In [9]:
import os
import pandas as pd
import openpyxl

# Define the directory where your CSV files are located
csv_directory = 'csvs'  # Change this to the directory containing your CSV files

# Define the directory where you want to save the filtered CSV files
output_csv_directory = 'output_csv'  # Change this to the directory where you want to save the filtered CSV files

# Create the output directory for CSV files if it doesn't exist
os.makedirs(output_csv_directory, exist_ok=True)

# Define the directory where you want to save the Excel file
output_excel_directory = 'output_excel'  # Change this to the directory where you want to save the Excel file

# Create the output directory for the Excel file if it doesn't exist
os.makedirs(output_excel_directory, exist_ok=True)

# Specify the Excel file path
excel_file_path = 'GENeSYS-MOD_User_Input_Settings_v01_phe_05-09-2023.xlsx'  # Replace with the path to your Excel file

df = pd.read_excel(excel_file_path, engine='openpyxl')


filtered_df = df[df.iloc[:, 4] == 1] 


unique_values = filtered_df.iloc[:, 3].unique() 

filtered_df.head()


csvs


,Continent,Country,Country_Sublevel_1,Country_Sublevel_2,Region Selected
0,Europe,DE,DE-BY,DE-BY-UF,1
1,Europe,NO,NO1,NO11,1
2,Europe,NO,NO1,NO1,1
4,Europe,DE,DE-BY,DE-BY,1
7,Asia,CN,CN-Cen,CN-Cen,1


In [7]:
# Initialize an empty dictionary to store DataFrames
data_frames = {}

In [29]:
# Initialize the Excel writer
output_excel_file_path = os.path.join(output_excel_directory, 'output.xlsx')
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv'):
            # Construct the full path to the CSV file
            csv_file_path = os.path.join(csv_directory, filename)
            
            df_first = pd.read_csv(csv_file_path, delimiter=';',header=None)
            df_header = df_first.iloc[:3] #shweta
            df_header = df_header.apply(lambda x: x.apply(lambda y: str(y).replace('.0', '')))

            
            # Read the CSV file into a Pandas DataFrame, skipping the first 4 rows
            #df = pd.read_csv(csv_file_path, skiprows=4, delimiter=';')
            
            df = df_first.iloc[3:]
            
            # Extract values from the first column
            first_column_values = df.iloc[:, 0]
            
            # Filter the DataFrame based on whether the values in the first column are present in unique_values_from_first_df
            filtered_df = df[first_column_values.isin(unique_values)]
            df_final = pd.concat([df_header, filtered_df], axis=0)
            
            df_final.replace('nan', '', inplace=True)
            s
            
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(filename)[0]
            
            # Store the DataFrame in the dictionary with the filename (without extension) as the key
            data_frames[worksheet_name] = df_final
            
            # Write the DataFrame to an Excel worksheet with the same filename (without extension)
            df_final.to_excel(writer, sheet_name=worksheet_name, index=False, header=False)
            
            # Construct the full path to the output CSV file
            output_csv_file_path = os.path.join(output_csv_directory, filename)
            
            # Save the filtered DataFrame to the CSV file without the index
            df_final.to_csv(output_csv_file_path, index=False, header=False)